<a href="https://colab.research.google.com/github/salvatore-dacunto/beam_demo/blob/master/Beam_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install apache-beam[interactive]

#Movie Ratings

In [9]:
# import apache beam
import apache_beam as beam

# create pipeline
p1 = beam.Pipeline()

rating_4 = (
    p1
    | beam.io.ReadFromText('movies_rating.txt', skip_header_lines = 1) # read movie rating file, skip header
    | beam.Map(lambda record: record.split(',')) # split the records by comma delimeter
    | beam.Filter(lambda record: float(record[2]) > 4) # filter for all records with a rating greater than 4
    | beam.io.WriteToText('rating_4.txt') # write the results to file
)

p1.run()

In [3]:
! cat /content/rating_4.txt-00000-of-00001

['3', 'SpiderMan', '4.5']


#Word Counts

In [19]:
words = ['peter', 'piper', 'picked', 'peck', 'pepper']

def find_word(element):
  if element in words:
    return True

p2 = beam.Pipeline()

results = (
    p2
    | beam.io.ReadFromText('Peter_Piper.txt')
    | beam.FlatMap(lambda record: record.split(' '))
    | beam.Filter(find_word)
    | beam.Map(lambda record: (record,1))
    | beam.CombinePerKey(sum)
    | beam.io.WriteToText('PP_results.txt')
)

p2.run()

In [20]:
! cat /content/PP_results.txt-00000-of-00001

('peter', 4)
('piper', 4)
('picked', 4)
('peck', 4)
('pepper', 4)


# Customer Data Using Pardo

In [25]:
class SplitRow(beam.DoFn):
  def process(self,element):
    return [element.split(',')]

class FilterCustomer(beam.DoFn):
  def process(self,element):
    if (element[2]) == 'NY' and (int(element[3]) > 20):
      return [element]

p3 = beam.Pipeline()
result = (
    p3
    | 'Read your input file' >> beam.io.ReadFromText('Customers_age.txt')
    | beam.ParDo(SplitRow())
    | beam.ParDo(FilterCustomer())
    | beam.io.WriteToText('customer_result.txt')
)
p3.run()

In [26]:
! cat /content/customer_result.txt-00000-of-00001

['1', 'John', 'NY', '22']
['3', 'Mary', 'NY', '30']
